In [1]:
import ultralytics
import torch
import easyocr

import numpy as np
import time
from concurrent.futures import ThreadPoolExecutor
from collections import defaultdict
from threading import Lock
from datetime import datetime
import pyodbc

In [2]:
from ultralytics import YOLO
from ultralytics.utils.plotting import Annotator, colors

In [3]:
import cv2

In [20]:
model1 = YOLO("yolov8n")
model2 = YOLO("license_plate_detector (1).pt")

In [5]:
reader = easyocr.Reader(['en'],gpu=False)

Using CPU. Note: This module is much faster with a GPU.


In [42]:
license_plate_data = []
vehicle_tracker = {}  # Dictionary to keep track of vehicles
next_vehicle_id = 1
lock = Lock()

In [43]:
vehicle_classes = ['car', 'truck', 'bus', 'motorcycle', 'bicycle']

In [44]:

def infer_frame(frame, model, with_tracking=False):
    if with_tracking:
        results = model.track(frame, persist=True)
    else:
        results = model(frame)
    return results

In [45]:
def process_frame(frame):
    with ThreadPoolExecutor() as executor:
        future1 = executor.submit(infer_frame, frame, model1, True)  # Enable tracking for model1
        future2 = executor.submit(infer_frame, frame, model2, True)  # Keep tracking for model2
        results1 = future1.result()
        results2 = future2.result()
        return results1, results2

In [46]:
def perform_ocr_on_plate(frame, box):
    x1, y1, x2, y2 = map(int, box.xyxy[0])
    cropped_plate = frame[y1:y2, x1:x2]
    ocr_result = reader.readtext(cropped_plate)

    if len(ocr_result) > 0:
        text = ocr_result[0][-2]
        return text
    return None

In [ ]:
connection_string = "DRIVER={SQL Server};SERVER=DESKTOP-SS2CQM3\SQLEXPRESS02;DATABASE=yourdatabasename;UID=sa;PWD=yourpassword"
conn = pyodbc.connect(connection_string)
cursor = conn.cursor()

In [48]:
def create_table():
    cursor.execute("""
    IF NOT EXISTS (SELECT * FROM sysobjects WHERE name='license_plate_data' AND xtype='U')
    CREATE TABLE license_plate_data (
        detection_time DATETIME,
        unique_id INT,
        vehicle_class VARCHAR(50),
        license_plate_text VARCHAR(20)
    )
    """)
    conn.commit()

In [49]:
def insert_data(data):
    for item in data:
        detection_time, unique_id, vehicle_class, license_plate_text = item
        
        # Check if the unique_id already exists in the database
        cursor.execute("SELECT COUNT(1) FROM license_plate_data WHERE unique_id = ?", (unique_id,))
        exists = cursor.fetchone()[0]
        
        # Only insert if the unique_id does not exist
        if not exists:
            cursor.execute("""
            INSERT INTO license_plate_data (detection_time, unique_id, vehicle_class, license_plate_text)
            VALUES (?, ?, ?, ?)
            """, (detection_time, unique_id, vehicle_class, license_plate_text))
            conn.commit()
        else:
            print(f"Skipping duplicate entry for unique_id: {unique_id}")


In [50]:
def draw_results(frame, results1, results2):
    global next_vehicle_id, vehicle_tracker
    annotator = Annotator(frame, line_width=3, font_size=18)

    vehicle_boxes = {}
    for result in results1:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            class_id = int(box.cls[0])
            track_id = int(box.id[0]) if box.id is not None else None

            vehicle_class = model1.names[class_id]

            if vehicle_class in vehicle_classes:
                if track_id is not None:
                    if track_id not in vehicle_tracker:
                        with lock:
                            vehicle_tracker[track_id] = next_vehicle_id
                            next_vehicle_id += 1

                    unique_id = vehicle_tracker[track_id]
                    vehicle_boxes[unique_id] = (x1, y1, x2, y2, vehicle_class)

                    label = f'{vehicle_class.upper()} {conf:.2f} ID: {unique_id}'
                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                    cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), (0, 255, 0), -1)
                    annotator.box_label((x1, y1, x2, y2), label, color=(0, 0, 0))

    for result in results2:
        boxes = result.boxes
        for box in boxes:
            x1, y1, x2, y2 = map(int, box.xyxy[0])
            conf = box.conf[0]
            track_id = int(box.id[0]) if box.id is not None else None

            if track_id is not None:
                plate_text = perform_ocr_on_plate(frame, box)
                if plate_text:
                    closest_vehicle = min(vehicle_boxes.items(), key=lambda item:
                        abs(item[1][0] - x1) + abs(item[1][1] - y1))
                    vehicle_id, (vx1, vy1, vx2, vy2, vehicle_class) = closest_vehicle

                    current_time = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
                    license_plate_data.append([current_time, vehicle_id, vehicle_class, plate_text])
                    label = f'Plate: {plate_text}'
                    print(f"Detected License Plate: {plate_text}, Vehicle: {vehicle_class}, Unique ID: {vehicle_id} at time {current_time}")

                    label_size = cv2.getTextSize(label, cv2.FONT_HERSHEY_SIMPLEX, 0.7, 2)[0]
                    cv2.rectangle(frame, (x1, y1 - label_size[1] - 10), (x1 + label_size[0], y1), (255, 0, 0), -1)
                    annotator.box_label((x1, y1, x2, y2), label, color=(255, 0, 0))

    return frame


In [51]:
## yah ek entry ko many time store krta ha
if __name__ == "__main__":
    create_table()

    input_video = 'new.mp4'
    video_capture = cv2.VideoCapture(input_video)
    fps = video_capture.get(cv2.CAP_PROP_FPS)
    width = int(video_capture.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(video_capture.get(cv2.CAP_PROP_FRAME_HEIGHT))

    license_plate_data = []
    i = 0
    while True:
        ret, frame = video_capture.read()
        if not ret:
            break
        
        # Process the frame (e.g., detection and results)
        results1, results2 = process_frame(frame)
        frame = draw_results(frame, results1, results2)  # Annotate the frame

        # Display the frame using OpenCV's imshow
        frame = cv2.resize(frame,(700,500))
        cv2.imshow('Vehicle Detection', frame)

        # Increment frame counter
        i += 1

        # Press 'q' to exit the video early
        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    # Insert collected data into the database
    insert_data(license_plate_data)

    # Release video capture and close display windows
    video_capture.release()
    cv2.destroyAllWindows()
    conn.close()



0: 640x384 1 person, 4 cars, 1 truck, 223.9ms
Speed: 4.0ms preprocess, 223.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 2 license_plates, 239.5ms
Speed: 1.0ms preprocess, 239.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)


0: 640x384 1 license_plate, 248.0ms
Speed: 15.0ms preprocess, 248.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 person, 4 cars, 1 truck, 259.0ms
Speed: 45.1ms preprocess, 259.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: 5379MX-B, Vehicle: car, Unique ID: 4 at time 2024-11-04 18:15:32


0: 640x384 1 person, 4 cars, 1 truck, 217.3ms
Speed: 18.0ms preprocess, 217.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 license_plate, 214.4ms
Speed: 20.8ms preprocess, 214.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: 5379MX-L, Vehicle: car, Unique ID: 4 at ti


0: 640x384 2 license_plates, 198.7ms
Speed: 7.9ms preprocess, 198.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 person, 4 cars, 1 truck, 201.7ms
Speed: 20.5ms preprocess, 201.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: BY, Vehicle: car, Unique ID: 2 at time 2024-11-04 18:15:50
Detected License Plate: 3066MP-, Vehicle: car, Unique ID: 1 at time 2024-11-04 18:15:51


0: 640x384 2 license_plates, 197.2ms
Speed: 8.0ms preprocess, 197.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 person, 4 cars, 1 truck, 213.8ms
Speed: 17.7ms preprocess, 213.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: 5379MX-L, Vehicle: car, Unique ID: 2 at time 2024-11-04 18:15:52
Detected License Plate: 3066MP-L, Vehicle: car, Unique ID: 1 at time 2024-11-04 18:15:52


0: 640x384 2 license_plates, 193.8ms
Speed: 6.4ms preprocess, 193.8ms inference, 15.6m

Detected License Plate: 3066MP-4, Vehicle: car, Unique ID: 1 at time 2024-11-04 18:16:07


0: 640x384 1 license_plate, 193.0ms
Speed: 9.2ms preprocess, 193.0ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 person, 4 cars, 1 truck, 195.2ms
Speed: 25.9ms preprocess, 195.2ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: 3066MP-4, Vehicle: car, Unique ID: 1 at time 2024-11-04 18:16:07


0: 640x384 1 license_plate, 210.3ms
Speed: 10.4ms preprocess, 210.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 person, 4 cars, 1 truck, 198.7ms
Speed: 22.0ms preprocess, 198.7ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: 3066MP-4, Vehicle: car, Unique ID: 1 at time 2024-11-04 18:16:08


0: 640x384 1 license_plate, 196.2ms
Speed: 8.0ms preprocess, 196.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 person, 4 cars, 1 


0: 640x384 1 person, 3 cars, 1 truck, 202.9ms
Speed: 19.4ms preprocess, 202.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 license_plate, 197.9ms
Speed: 7.0ms preprocess, 197.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: 3066MP-4, Vehicle: car, Unique ID: 3 at time 2024-11-04 18:16:21


0: 640x384 1 license_plate, 194.8ms
Speed: 6.2ms preprocess, 194.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 person, 3 cars, 1 truck, 198.0ms
Speed: 21.2ms preprocess, 198.0ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: 3066MP-4, Vehicle: car, Unique ID: 3 at time 2024-11-04 18:16:22


0: 640x384 1 license_plate, 196.2ms
Speed: 10.3ms preprocess, 196.2ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 person, 3 cars, 1 truck, 200.2ms
Speed: 21.9ms preprocess, 200.2ms inference, 15.6ms postprocess per image at

0: 640x384 1 person, 2 cars, 1 truck, 203.8ms
Speed: 43.1ms preprocess, 203.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: 3066MP-4, Vehicle: car, Unique ID: 2 at time 2024-11-04 18:16:37


0: 640x384 1 license_plate, 190.1ms
Speed: 11.2ms preprocess, 190.1ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 person, 2 cars, 1 truck, 205.7ms
Speed: 26.8ms preprocess, 205.7ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: BY, Vehicle: car, Unique ID: 2 at time 2024-11-04 18:16:38


0: 640x384 1 license_plate, 190.5ms
Speed: 9.3ms preprocess, 190.5ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 person, 2 cars, 1 truck, 211.5ms
Speed: 21.1ms preprocess, 211.5ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: BY, Vehicle: car, Unique ID: 2 at time 2024-11-04 18:16:38


0: 640x384 1 license_plate, 208.0m


0: 640x384 1 license_plate, 195.3ms
Speed: 9.1ms preprocess, 195.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 2 persons, 2 cars, 1 truck, 210.9ms
Speed: 24.7ms preprocess, 210.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: AP L895.5, Vehicle: car, Unique ID: 3 at time 2024-11-04 18:16:54


0: 640x384 1 license_plate, 194.9ms
Speed: 7.0ms preprocess, 194.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 2 persons, 2 cars, 1 truck, 197.9ms
Speed: 21.2ms preprocess, 197.9ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: AP 489555, Vehicle: car, Unique ID: 3 at time 2024-11-04 18:16:54


0: 640x384 1 license_plate, 191.2ms
Speed: 7.3ms preprocess, 191.2ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 2 persons, 2 cars, 1 truck, 211.1ms
Speed: 8.9ms preprocess, 211.1ms inference, 0.0ms postprocess per image a

Speed: 7.0ms preprocess, 211.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 2 persons, 1 truck, 215.0ms
Speed: 21.6ms preprocess, 215.0ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: AP L895 5, Vehicle: truck, Unique ID: 5 at time 2024-11-04 18:17:07


0: 640x384 1 license_plate, 205.2ms
Speed: 9.2ms preprocess, 205.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 2 persons, 1 truck, 206.2ms
Speed: 28.4ms preprocess, 206.2ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: AP 4895-5, Vehicle: truck, Unique ID: 5 at time 2024-11-04 18:17:08


0: 640x384 1 license_plate, 194.3ms
Speed: 8.0ms preprocess, 194.3ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 2 persons, 1 truck, 194.3ms
Speed: 25.9ms preprocess, 194.3ms inference, 15.6ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: AP 4895

Speed: 37.3ms preprocess, 199.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: AP 4895-5, Vehicle: truck, Unique ID: 5 at time 2024-11-04 18:17:21


0: 640x384 2 persons, 1 truck, 199.4ms
Speed: 21.3ms preprocess, 199.4ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 1 license_plate, 209.6ms
Speed: 8.5ms preprocess, 209.6ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: AP 489555, Vehicle: truck, Unique ID: 5 at time 2024-11-04 18:17:21


0: 640x384 1 license_plate, 195.8ms
Speed: 8.0ms preprocess, 195.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
0: 640x384 2 persons, 1 truck, 200.8ms
Speed: 20.0ms preprocess, 200.8ms inference, 0.0ms postprocess per image at shape (1, 3, 640, 384)
Detected License Plate: AP L895-5, Vehicle: truck, Unique ID: 5 at time 2024-11-04 18:17:22


0: 640x384 1 license_plate, 194.4ms
Speed: 8.0ms preprocess, 194.4ms infere